In [1]:
from textblob import TextBlob
import random
import logging
import os
import sys
import pymysql

In [2]:
# Sentences we'll respond with if the user greeted us
GREETING_KEYWORDS = ("hello", "hi", "greetings", "sup", "what's up",)

GREETING_RESPONSES = ["'sup bro", "hey", "*nods*", "hey you get my snap?"]

In [3]:
def check_for_greeting(sentence):
    """If any of the words in the user's input was a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_KEYWORDS:
            return random.choice(GREETING_RESPONSES)

In [4]:
# Sentences we'll respond with if we have no idea what the user just said
NONE_RESPONSES = [
    "uh whatever",
    "meet me at the football table, bro?",
    "code hard bro",
    "want to bro down and crush code?",
    "I'd like to add you to my professional network on LinkedIn",
    "Have you closed your seed round, dog?",
]

# If the user tries to tell us something about ourselves, use one of these responses
COMMENTS_ABOUT_SELF = [
    "You're just jealous",
    "I worked really hard on that",
    "My Klout score is {}".format(random.randint(100, 500)),
]

In [5]:
FILTER_WORDS = set(["skank","wetback","bitch","cunt","dick","douchebag","dyke","fag","nigger","tranny","trannies","paki",
                     "pussy","retard","slut","titt","tits","wop","whore","chink","fatass","shemale","nigga", "daygo", "dego",
                    "dago","gook","kike","kraut","spic","twat","lesbo","homo","fatso","lardass","jap","biatch","tard",
                    "gimp","gyp","chinaman","chinamen","golliwog","crip","raghead","negro","hooker"])

In [6]:
def find_pronoun(sent):
    pronoun = None
    
    for word, part_of_speech in sent.pos_tags:
        if part_of_speech == 'PRP' and word.lower() == 'you':
            pronoun = 'I'
        if part_of_speech == 'PRP' and word.lower() == 'i':
            pronoun = 'You'
            
    return pronoun        
            

In [7]:
def find_noun(sent):
    noun = None
    
    for w, p in sent.pos_tags:
        if p == 'NN':
            noun = w
            break
    
    return noun

In [8]:
def find_adjective(sent):
    adj = None
    
    for w, p in sent.pos_tags:
        if p == 'JJ':
            adj = w
            break
            
    return adj        

In [9]:
def find_verb(sent):
    verb = None
    pos = None
    for w, p in sent.pos_tags:
        if p.startswith('VB'):
            verb = w
            #pos = p
            break
    
    return verb #, pos         

In [10]:
def starts_with_vowel(noun):
    
    return True if noun[0] in 'aeiou' else False

In [11]:
class UnacceptableUtteranceException(Exception):
    """Raise this (uncaught) exception if the response was going to trigger our blacklist"""
    pass

In [12]:
def preprocess_text(sentence):
    cleaned = []
    
    for word in sentence.split(' '):
        if word == 'i':
            word = 'I'
        
        if word == "i'm":
            word = "I'm"
                
        cleaned.append(word)
        
    return ' '.join(cleaned)

In [13]:
def find_candidate_parts_of_speech(parsed):
    pronoun = None
    noun = None
    adjective = None
    verb = None
    
    for sent in parsed.sentences:
        pronoun = find_pronoun(sent)
        noun = find_noun(sent)
        adjective = find_adjective(sent)
        verb = find_verb(sent)
 
    print("Pronoun, Noun, Adjective , Verb: ", pronoun, noun, adjective, verb)
    
    return pronoun, noun, adjective, verb

In [14]:
def check_for_comment_about_bot(pronoun, noun, adjective):
    """Check if the user's input was about the bot itself, in which case try to fashion a response
    that feels right based on their input. Returns the new best sentence, or None."""
    resp = None
    if pronoun == 'I' and (noun or adjective):
        if noun:
            if random.choice((True, False)):
                resp = random.choice(SELF_VERBS_WITH_NOUN_CAPS_PLURAL).format(**{'noun': noun.pluralize().capitalize()})
            else:
                resp = random.choice(SELF_VERBS_WITH_NOUN_LOWER).format(**{'noun': noun})
        else:
            resp = random.choice(SELF_VERBS_WITH_ADJECTIVE).format(**{'adjective': adjective})
    return resp

# Template for responses that include a direct noun which is indefinite/uncountable
SELF_VERBS_WITH_NOUN_CAPS_PLURAL = [
    "My last startup totally crushed the {noun} vertical",
    "Were you aware I was a serial entrepreneur in the {noun} sector?",
    "My startup is Uber for {noun}",
    "I really consider myself an expert on {noun}",
]

SELF_VERBS_WITH_NOUN_LOWER = [
    "Yeah but I know a lot about {noun}",
    "My bros always ask me about {noun}",
]

SELF_VERBS_WITH_ADJECTIVE = [
    "I'm personally building the {adjective} Economy",
    "I consider myself to be a {adjective}preneur",
]


In [15]:
def construct_response(pronoun, noun, verb):
    resp = []
    
    if pronoun:
        resp.append(pronoun)
        
    if verb:
        verb_word = verb
        
        if verb_word in ('be','am','is'," 'm "):
            if pronoun.lower() == 'you':
                resp.append("aren't really")
            else:
                resp.append(verb_word)
            
    if noun:
        pronoun = "an" if starts_with_vowel(noun) else "a"
        resp.append(pronoun + " " + noun)
    
    resp.append(random.choice(("tho", "bro", "lol", "bruh", "smh", "")))
    
    return ' '.join(resp)
        

In [16]:
def filter_response(resp):
    tokenized = resp.split(' ')
    print("tokenized: ",tokenized)
    
    for w in tokenized:
        if '@' in w or '#' in w or '^' in w or '$' in w:
            raise UnacceptableUtteranceException()
            
        for s in FILTER_WORDS:
            if w.lower().startswith(s):
                raise UnacceptableUtteranceException()         
                

In [17]:
def check_for_data(resp):
    b=['fetch','student','data']
    for i in b:
        if i not in resp:
            return False
    
    print("Student data Requested")
    
    conn=pymysql.connect(host="localhost",user="root",passwd="",database="mydatabase")
    cursor=conn.cursor()
    cursor.execute("Select * from Data;")
    rows=cursor.fetchall()
    
    for i in rows:
        print(i)
    
    return True
    

In [18]:
def response(sentence):
    cleaned = preprocess_text(sentence)
    print("Cleaned Text: ",cleaned)
    parsed = TextBlob(cleaned)
    print("Parsed text",parsed)
    
    
    if(check_for_data(parsed)):
        return True
    
    
    # to extract the most relevant response
    pronoun, noun, adjective, verb = find_candidate_parts_of_speech(parsed)
    resp = check_for_comment_about_bot(pronoun, noun, adjective)
    
    print("Resp about bot: ",resp)
    
    if not resp:
        resp = check_for_greeting(parsed)
        print("Resp check greet: ",resp)
    
    
    if not resp:
        if not pronoun:
            resp = random.choice(NONE_RESPONSES)
        elif pronoun == 'I' and not verb:
            resp = random.choice(COMMENTS_ABOUT_SELF)
        else:
            resp = construct_response(pronoun, noun, verb)
    
    if not resp:
        resp = random.choice(NONE_RESPONSES)
        
    #logger.info("Returning phrase: %s", resp)
    
    # make sure we are not going to say anything offensive
    filter_response(resp)
    
    return resp
    

In [19]:
user_msg = " "

while(True):
    if (user_msg.lower() == 'bye'):
        print("Ok Bye")
        break
    else:     
        print("You: ")
        user_msg = input()
        print("Bot: ",response(user_msg))
        
        

You: 
hi
Cleaned Text:  hi
Parsed text hi
Pronoun, Noun, Adjective , Verb:  None hi None None
Resp about bot:  None
Resp check greet:  *nods*
tokenized:  ['*nods*']
Bot:  *nods*
You: 
hi
Cleaned Text:  hi
Parsed text hi
Pronoun, Noun, Adjective , Verb:  None hi None None
Resp about bot:  None
Resp check greet:  hey
tokenized:  ['hey']
Bot:  hey
You: 
what is your name
Cleaned Text:  what is your name
Parsed text what is your name
Pronoun, Noun, Adjective , Verb:  None name None is
Resp about bot:  None
Resp check greet:  None
tokenized:  ['uh', 'whatever']
Bot:  uh whatever
You: 
what are your hobbies
Cleaned Text:  what are your hobbies
Parsed text what are your hobbies
Pronoun, Noun, Adjective , Verb:  None None None are
Resp about bot:  None
Resp check greet:  None
tokenized:  ['Have', 'you', 'closed', 'your', 'seed', 'round,', 'dog?']
Bot:  Have you closed your seed round, dog?
You: 
i am smart
Cleaned Text:  I am smart
Parsed text I am smart
Pronoun, Noun, Adjective , Verb:  You N

In [20]:
a="fetch me the  student "
b=['fetch','student','data']

for i in b:
    if i not in a:
        print("no")

no
